In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('recommender').getOrCreate()

In [2]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [6]:
path = '/home/jovyan/work/original/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Recommender_Systems/movielens_ratings.csv'
df = spark.read.csv(path, inferSchema=True, header=True)

In [8]:
df.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [10]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [11]:
training_set, test_set = df.randomSplit([0.8, 0.2])

In [13]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')
model = als.fit(training_set)

In [15]:
predictions = model.transform(test_set)
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   1.0|     4| 0.6206041|
|     31|   1.0|    29| 1.1653287|
|     65|   1.0|    22| 1.9962176|
|     65|   2.0|    15| 0.3650776|
|     53|   1.0|     6| 2.8255696|
|     53|   1.0|    23| -0.993066|
|     78|   1.0|    22| 0.8881082|
|     78|   1.0|     1|0.78987724|
|     78|   1.0|    19|0.38776535|
|     78|   1.0|     4| 0.9614343|
|     81|   1.0|     1| 1.3535336|
|     81|   1.0|    15| 2.0138605|
|     81|   2.0|     9|  4.683103|
|     81|   1.0|    21|  2.945009|
|     28|   1.0|    17| 2.0456424|
|     28|   5.0|    18| 1.6587181|
|     76|   1.0|    20| 0.9513607|
|     26|   1.0|     3| 1.8958751|
|     26|   1.0|     7| 3.4177077|
|     27|   1.0|    13|0.26360413|
+-------+------+------+----------+
only showing top 20 rows



In [19]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating')
rmse = evaluator.evaluate(predictions)
print(f"RMSE: {rmse}")

RMSE: 1.7631326093141162


In [22]:
single_user = test_set.filter(test_set['userId']==11).select(['movieId', 'userId'])
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     20|    11|
|     25|    11|
|     30|    11|
|     40|    11|
|     41|    11|
|     45|    11|
|     47|    11|
|     80|    11|
|     88|    11|
+-------+------+



In [23]:
recommendations = model.transform(single_user)
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     30|    11| 3.7159657|
|     88|    11|  2.478394|
|     45|    11| 1.8636799|
|     47|    11| 1.5006026|
|     40|    11| 1.2935965|
|     80|    11| 1.0259826|
|     41|    11| 0.8284821|
|     25|    11| 0.7391672|
|      9|    11|0.54812396|
|     20|    11|0.30911398|
+-------+------+----------+

